In [2]:
import torch
from torch_geometric.data import Data
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

import numpy as np
import pandas as pd
import networkx as nx

import meshio

In [96]:
mesh_file = "meshes/hole_middle_order1.inp"
csv_file = "data/test_data2.csv"

mesh = meshio.read(mesh_file)
df = pd.read_csv(csv_file, sep=',', skipinitialspace=True)

In [110]:
class Geometry():
    def __init__(self, mesh_file, data_file):

        self.mesh = meshio.read(mesh_file)
        self.node_pos = torch.tensor(self.mesh.points[:, :2], dtype=torch.float)
        self.mesh_size = self.mesh.points.shape[0]

        self.df = pd.read_csv(data_file, sep=',', skipinitialspace=True)
        self.df = self.df.iloc[:self.mesh_size]

        self.create_dataset()


    def create_dataset(self):
        von_miss = self.df["Equivalent Stress"].values
        self.node_attr = torch.tensor(von_miss, dtype=torch.float).view(-1, 1)


    def create_graph(self):
        edges = []
        for cell in self.mesh.cells:
            if cell.type == "triangle":
                for element in cell.data:
                    edges.append([element[0], element[1]])
                    edges.append([element[1], element[0]])

                    edges.append([element[1], element[2]])
                    edges.append([element[2], element[1]])

                    edges.append([element[2], element[0]])
                    edges.append([element[0], element[2]])

        edges = torch.tensor(edges, dtype=torch.long).t().contiguous()
        
        self.graph = Data(x=self.node_attr, edge_index=edges, pos=self.node_pos)

        print(self.graph)
        

In [112]:
obj = Geometry(mesh_file, csv_file)

obj.create_graph()

Data(x=[5529, 1], edge_index=[2, 64176], pos=[5529, 2])


In [113]:
print(obj.graph.x)

tensor([[83749144.],
        [49995064.],
        [49995992.],
        ...,
        [79559056.],
        [31994600.],
        [38777104.]])
